# Create labeled results from exported spreadsheet
To use as truth for evaluating various ranking algorithms.

Labeled_results_raw contains the CSV export of the spreadsheet where unlabeled results were labeled.

Labeled_results contains the labeled results, ready to be used for evaluation.

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [ ]:
import json
import os

import pandas as pd

**First:** download the "Rate Results" tab of the rating spreadsheet to {input_dir}/{filename}.csv

**Second:** remove any instructions rows above the headers

In [ ]:
input_dir = '../data/rank_eval/labeled_results_raw'
output_dir = '../data/rank_eval/labeled_results'
filename = '2023-04-25'

## Read raw labeled results

In [ ]:
# read csv file into pandas
results_df = pd.read_csv(os.path.join(input_dir, f"{filename}.csv"))
results_df = results_df.drop(["Number"], axis=1, errors="ignore")
results_df = results_df.rename(columns={results_df.columns[1]: 'score'})

In [ ]:
results_df.head(3)

## Group by query and keep only results with one or more scores > 0

In [ ]:
results = [{'query': query, 'results': [{'id': str(row.result), 
                                        'score': row.score, 
                                        'text': row.text} \
                             for _, row in results.iterrows() if row.score > 0.0]} \
           for query, results in results_df.groupby('query')]
results = [result for result in results if len(result['results']) > 0]

In [ ]:
print(len(results))
results[0]

## Save labeled results

In [ ]:
with open(os.path.join(output_dir, f"{filename}.json"), 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)